In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:36627")
client

<Client: 'tcp://127.0.0.1:36627' processes=8 threads=40, memory=157.36 GiB>

In [2]:
import numpy as np
import xarray as xr
import pandas as pd
import os.path as op
import os
import gsw
import xrft
import xwavelet
from scipy.interpolate import interp2d, bisplrep, bisplev, griddata
from xgcm.grid import Grid
import MITgcmutils as mitutils
from xmitgcm import open_mdsdataset
from fastjmd95 import rho as densjmd95
import matplotlib.colors as clr
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
ddir = '/tank/chaocean/qjamet/RUNS/ORAR/reruns2/'
gdir = '/tank/chaocean/grid_chaO/gridMIT_update1/'
savedir = '/tank/topog/tuchida/Wavelet-spectrum/'
spec = '/tank/spectre/tuchida/NA-wavelets/'

In [4]:
grav = 9.81
nensembs = 5
Kh = 20.
K4 = 1e10
Kr = 1e-5
thetaMax = 1e20
rhoConst = 9.998e2
# Cp = 3.994e3
Cp = 3989.244953
# years = np.arange(2008,2013,dtype=int)
ySstart = -15
yNstart = 15
yend = 50
dnf = 2160
nfile = 73

In [5]:
nmembs = np.arange(100,148)
xchunk = 200
ychunk = 200

In [12]:
# iiter = 1419120

for nmemb in nmembs:
    if nmemb == 100:
        ds = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                     % (nmemb)), 
                             grid_dir=gdir,
                             prefix=['diagUm'], delta_t=2e2,
                             iters=iiter, 
                             ignore_unknown_vars=True
                            ).reset_coords(drop=True).isel(time=0).drop_dims(['YG','XC','Zl','Zu','Zp1']).chunk({'YC':ychunk,'XG':xchunk,
                                                                          'Z':10})
        
    else:
        ds1 = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                     % (nmemb)), 
                             grid_dir=gdir,
                             prefix=['diagUm'], delta_t=2e2,
                             iters=iiter, 
                             ignore_unknown_vars=True
                            ).reset_coords(drop=True).isel(time=0).drop_dims(['YG','XC','Zl','Zu','Zp1']).chunk({'YC':ychunk,'XG':xchunk,
                                                                          'Z':10})
        
        ds = xr.concat([ds, ds1], dim='nmemb')
        ds1.close()
    print(nmemb)

            # ds = ds.chunk({'nmemb':6})
ds.coords['nmemb'] = ('nmemb',nmembs)
ds

100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147


,Array,Chunk
Bytes,14.81 GiB,3.05 MiB
Shape,"(48, 46, 900, 1000)","(1, 10, 200, 200)"
Count,158875 Tasks,6000 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,14.81 GiB,3.05 MiB
Shape,"(48, 46, 900, 1000)","(1, 10, 200, 200)"
Count,158875 Tasks,6000 Chunks
Type,float64,numpy.ndarray


In [14]:
dsave = ds.mean('nmemb',skipna=True)
dsave.encoding = {}
dsave.to_zarr(op.join(savedir,'SnapsUm_EnsMean_0001419123.zarr'), mode='w')
dsave.close()
ds.close()

In [8]:
years = np.arange(1967,1968,dtype=int)
ntimes = np.arange(1419120,1574640+dnf,dnf).reshape(len(years),nfile)  # 

dd = 50
print(ntimes[0,dd:])

[1527120 1529280 1531440 1533600 1535760 1537920 1540080 1542240 1544400
 1546560 1548720 1550880 1553040 1555200 1557360 1559520 1561680 1563840
 1566000 1568160 1570320 1572480 1574640]


In [9]:
# iiter = 1419120
# iiter = 1421280
iiter = 1423440

for iiter in ntimes[0,3:]:
    for nmemb in nmembs:
        if nmemb == 100:
            ds = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                         % (nmemb)), 
                                 grid_dir=gdir,
                                 prefix=['diagUm'], delta_t=2e2,
                                 iters=iiter, 
                                 ignore_unknown_vars=True
                                ).reset_coords(drop=True).drop_dims(['YG','XC','Zl','Zu','Zp1']
                                                                   ).chunk({'YC':ychunk,'XG':xchunk,
                                                                            'Z':10}).isel(time=0)
            Um_dEdx = xr.DataArray(mitutils.rdmds(op.join(ddir,'memb%03d/run1967/ocn/diagUm' 
                                                  % (nmemb)), itrs=iiter, 
                                                  rec=3,    # U from surface pressure
                              # usememmap=True
                                                 ),
                                   dims=ds.TOTUTEND.dims, coords=ds.TOTUTEND.coords
                                  ).chunk({'YC':ychunk,'XG':xchunk,'Z':10})

            Um_Disv = xr.DataArray(mitutils.rdmds(op.join(ddir,'memb%03d/run1967/ocn/diagUm' 
                                                          % (nmemb)), itrs=iiter, 
                                                  rec=5,    # Vertical dissipation
                                 # usememmap=True
                                                 ),
                                   dims=ds.TOTUTEND.dims, coords=ds.TOTUTEND.coords
                                  ).chunk({'XG':xchunk,'YC':ychunk,'Z':10})
        else:
            Um_dEdx = xr.concat([Um_dEdx, 
                                 xr.DataArray(mitutils.rdmds(op.join(ddir,
                                                                     'memb%03d/run1967/ocn/diagUm' 
                                                                     % (nmemb)), itrs=iiter, 
                                                             rec=3,    # U from surface pressure
                                                  # usememmap=True
                                                            ),
                                              dims=ds.TOTUTEND.dims, coords=ds.TOTUTEND.coords
                                             ).chunk({'YC':ychunk,'XG':xchunk,'Z':10})
                                ], dim='nmemb')
            Um_Disv = xr.concat([Um_Disv, 
                                 xr.DataArray(mitutils.rdmds(op.join(ddir,
                                                                     'memb%03d/run1967/ocn/diagUm' 
                                                                     % (nmemb)), itrs=iiter, 
                                                             rec=5,    # Vertical dissipation
                                                 # usememmap=True
                                                                             ),
                                              dims=ds.TOTUTEND.dims, coords=ds.TOTUTEND.coords
                                             ).chunk({'XG':xchunk,'YC':ychunk,'Z':10})
                                ], dim='nmemb')


                # ds = ds.chunk({'nmemb':6})
    Um_dEdx.coords['nmemb'] = ('nmemb',nmembs)
    Um_Disv.coords['nmemb'] = ('nmemb',nmembs)
    
    dsave = Um_dEdx.mean('nmemb',skipna=True).compute().chunk({'YC':ychunk,'XG':xchunk,'Z':10}
                                                             ).to_dataset(name='Um_dEdx')
    dsave['Um_Disv'] = Um_Disv.mean('nmemb',skipna=True).compute().chunk({'YC':ychunk,'XG':xchunk,
                                                                          'Z':10})
    dsave['Um_dEdx'].encoding = {}
    dsave['Um_Disv'].encoding = {}
    dsave.to_zarr(op.join(spec,'SnapsUm-ext_EnsMean/%010d.zarr' % iiter), mode='w')
    dsave.close()
    
    print(iiter)

1425600
1427760
1429920
1432080
1434240
1436400
1438560
1440720
1442880
1445040
1447200
1449360
1451520
1453680
1455840
1458000
1460160
1462320
1464480
1466640
1468800
1470960
1473120
1475280
1477440
1479600
1481760
1483920
1486080
1488240
1490400
1492560
1494720
1496880
1499040
1501200
1503360
1505520
1507680
1509840
1512000
1514160
1516320
1518480
1520640
1522800
1524960
1527120
1529280
1531440
1533600
1535760
1537920
1540080
1542240
1544400
1546560
1548720
1550880
1553040
1555200
1557360
1559520
1561680
1563840
1566000
1568160
1570320
1572480
1574640


In [11]:
dsave = Um_dEdx.mean('nmemb',skipna=True).compute().chunk({'YC':ychunk,'XG':xchunk,'Z':10}
                                                         ).to_dataset(name='Um_dEdx')
dsave['Um_Disv'] = Um_Disv.mean('nmemb',skipna=True).compute().chunk({'YC':ychunk,'XG':xchunk,
                                                                      'Z':10})
dsave['Um_dEdx'].encoding = {}
dsave['Um_Disv'].encoding = {}
dsave.to_zarr(op.join(savedir,'SnapsUm-ext_EnsMean_%010d.zarr' % iiter), mode='w')
dsave.close()

In [10]:
for nmemb in nmembs:
    if nmemb == 100:
        ds = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                     % (nmemb)), 
                             grid_dir=gdir,
                             prefix=['diagVm'], delta_t=2e2,
                             iters=1419123, 
                             ignore_unknown_vars=True
                            ).reset_coords(drop=True).isel(time=0).drop_dims(['YC','XG','Zl','Zu','Zp1']).chunk({'XC':xchunk,'YG':ychunk,
                                                                          'Z':10})
        
    else:
        ds1 = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                     % (nmemb)), 
                             grid_dir=gdir,
                             prefix=['diagVm'], delta_t=2e2,
                             iters=1419123, 
                             ignore_unknown_vars=True
                            ).reset_coords(drop=True).isel(time=0).drop_dims(['YC','XG','Zl','Zu','Zp1']).chunk({'XC':xchunk,'YG':ychunk,
                                                                          'Z':10})
        
        ds = xr.concat([ds, ds1], dim='nmemb')
        ds1.close()
    print(nmemb)

            # ds = ds.chunk({'nmemb':6})
ds.coords['nmemb'] = ('nmemb',nmembs)
ds

100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147


,Array,Chunk
Bytes,7.40 GiB,1.53 MiB
Shape,"(48, 46, 900, 1000)","(1, 10, 200, 200)"
Count,171067 Tasks,6000 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,7.40 GiB,1.53 MiB
Shape,"(48, 46, 900, 1000)","(1, 10, 200, 200)"
Count,171067 Tasks,6000 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [11]:
dsave = ds.chunk({'nmemb':4}).mean('nmemb',skipna=True)
dsave.encoding = {}
dsave.to_zarr(op.join(savedir,'SnapsVm_EnsMean_0001419123.zarr'), mode='w')
dsave.close()
ds.close()

In [12]:
# iiter = 1419120
# iiter = 1421280
iiter = 1423440

for iiter in ntimes[0,60:]:
    for nmemb in nmembs:
        if nmemb == 100:
            ds = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                         % (nmemb)), 
                                 grid_dir=gdir,
                                 prefix=['diagVm'], delta_t=2e2,
                                 iters=iiter, 
                                 ignore_unknown_vars=True
                                ).reset_coords(drop=True).drop_dims(['YC','XG','Zl','Zu','Zp1']
                                                                   ).chunk({'YG':ychunk,'XC':xchunk,
                                                                            'Z':10}).isel(time=0)
            Vm_dEdy = xr.DataArray(mitutils.rdmds(op.join(ddir,'memb%03d/run1967/ocn/diagVm' 
                                                  % (nmemb)), itrs=iiter, 
                                                  rec=3,    # U from surface pressure
                              # usememmap=True
                                                 ),
                                   dims=ds.TOTVTEND.dims, coords=ds.TOTVTEND.coords
                                  ).chunk({'YG':ychunk,'XC':xchunk,'Z':10})

            Vm_Disv = xr.DataArray(mitutils.rdmds(op.join(ddir,'memb%03d/run1967/ocn/diagVm' 
                                                          % (nmemb)), itrs=iiter, 
                                                  rec=5,    # Vertical dissipation
                                 # usememmap=True
                                                 ),
                                   dims=ds.TOTVTEND.dims, coords=ds.TOTVTEND.coords
                                  ).chunk({'XC':xchunk,'YG':ychunk,'Z':10})
        else:
            Vm_dEdy = xr.concat([Vm_dEdy, 
                                 xr.DataArray(mitutils.rdmds(op.join(ddir,
                                                                     'memb%03d/run1967/ocn/diagVm' 
                                                                     % (nmemb)), itrs=iiter, 
                                                             rec=3,    # U from surface pressure
                                                  # usememmap=True
                                                            ),
                                              dims=ds.TOTVTEND.dims, coords=ds.TOTVTEND.coords
                                             ).chunk({'YG':ychunk,'XC':xchunk,'Z':10})
                                ], dim='nmemb')
            Vm_Disv = xr.concat([Vm_Disv, 
                                 xr.DataArray(mitutils.rdmds(op.join(ddir,
                                                                     'memb%03d/run1967/ocn/diagVm' 
                                                                     % (nmemb)), itrs=iiter, 
                                                             rec=5,    # Vertical dissipation
                                                 # usememmap=True
                                                                             ),
                                              dims=ds.TOTVTEND.dims, coords=ds.TOTVTEND.coords
                                             ).chunk({'XC':xchunk,'YG':ychunk,'Z':10})
                                ], dim='nmemb')

        
    Vm_dEdy.coords['nmemb'] = ('nmemb',nmembs)
    Vm_Disv.coords['nmemb'] = ('nmemb',nmembs)
    
    dsave = Vm_dEdy.mean('nmemb',skipna=True).compute().chunk({'YG':ychunk,'XC':xchunk,'Z':10}
                                                             ).to_dataset(name='Vm_dEdy')
    dsave['Vm_Disv'] = Vm_Disv.mean('nmemb',skipna=True).compute().chunk({'YG':ychunk,'XC':xchunk,
                                                                          'Z':10})
    dsave['Vm_dEdy'].encoding = {}
    dsave['Vm_Disv'].encoding = {}
    dsave.to_zarr(op.join(spec,'SnapsVm-ext_EnsMean/%010d.zarr' % iiter), mode='w')
    dsave.close()
    
    print(iiter)

1548720
1550880
1553040
1555200
1557360
1559520
1561680
1563840
1566000
1568160
1570320
1572480
1574640


In [13]:
dsave = Vm_dEdy.mean('nmemb',skipna=True).compute().chunk({'YG':ychunk,'XC':xchunk,'Z':10}
                                                         ).to_dataset(name='Vm_dEdy')
dsave['Vm_Disv'] = Vm_Disv.mean('nmemb',skipna=True).compute().chunk({'YG':ychunk,'XC':xchunk,
                                                                      'Z':10})
dsave['Vm_dEdy'].encoding = {}
dsave['Vm_Disv'].encoding = {}
dsave.to_zarr(op.join(savedir,'SnapsVm-ext_EnsMean_%010d.zarr' % iiter), mode='w')
dsave.close()

In [8]:
for nmemb in nmembs:
    if nmemb == 100:
        ds = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                     % (nmemb)), 
                             grid_dir=gdir,
                             prefix=['snap_diagOcn'], delta_t=2e2,
                             iters=1419123, 
                             ignore_unknown_vars=True
                            ).isel(time=0).chunk({'XC':xchunk,'YC':ychunk,'YG':ychunk,'XG':xchunk})
    else:
        ds1 = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                     % (nmemb)), 
                             grid_dir=gdir,
                             prefix=['snap_diagOcn'], delta_t=2e2,
                             iters=1419123, 
                             ignore_unknown_vars=True
                            ).isel(time=0).chunk({'XC':xchunk,'YC':ychunk,'YG':ychunk,'XG':xchunk})
        ds = xr.concat([ds, ds1], dim='nmemb')
        ds1.close()
    print(nmemb)

            # ds = ds.chunk({'nmemb':6})
ds.coords['nmemb'] = ('nmemb',nmembs)
ds

100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147


<xarray.Dataset>
Dimensions:  (XC: 1000, YC: 900, XG: 1000, YG: 900, Z: 46, Zp1: 47, Zu: 46, Zl: 46, nmemb: 48)
Coordinates: (12/34)
    iter     int64 1419123
    time     timedelta64[ns] 3285 days 00:10:00
  * XC       (XC) >f4 262.0 262.1 262.2 262.3 262.3 ... 347.7 347.8 347.9 348.0
  * YC       (YC) >f4 -19.92 -19.83 -19.75 -19.67 ... 54.75 54.83 54.92 55.0
  * XG       (XG) >f4 262.0 262.0 262.1 262.2 262.3 ... 347.7 347.8 347.8 347.9
  * YG       (YG) >f4 -19.96 -19.87 -19.79 -19.71 ... 54.71 54.79 54.88 54.96
    ...       ...
    maskS    (Z, YG, XC) bool dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    maskInC  (YC, XC) bool dask.array<chunksize=(100, 100), meta=np.ndarray>
    rhoRef   (Z) >f4 dask.array<chunksize=(46,), meta=np.ndarray>
    maskInW  (YC, XG) bool dask.array<chunksize=(100, 100), meta=np.ndarray>
    maskInS  (YG, XC) bool dask.array<chunksize=(100, 100), meta=np.ndarray>
  * nmemb    (nmemb) int64 100 101 102 103 104 105 ... 142 143 144 145 146 147
Data variables:
    THETA    (nmemb, Z, YC, XC) float32 dask.array<chunksize=(1, 46, 100, 100), meta=np.ndarray>
    SALT     (nmemb, Z, YC, XC) float32 dask.array<chunksize=(1, 46, 100, 100), meta=np.ndarray>
    UVEL     (nmemb, Z, YC, XG) float32 dask.array<chunksize=(1, 46, 100, 100), meta=np.ndarray>
    VVEL     (nmemb, Z, YG, XC) float32 dask.array<chunksize=(1, 46, 100, 100), meta=np.ndarray>
    WVEL     (nmemb, Zl, YC, XC) float32 dask.array<chunksize=(1, 46, 100, 100), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    title:        netCDF wrapper of MITgcm MDS binary data
    source:       MITgcm
    history:      Created by calling `open_mdsdataset(grid_dir='/tank/chaocea...

In [10]:
dsave = ds.mean('nmemb',skipna=True)
dsave.encoding = {}
dsave.to_zarr(op.join(savedir,'Snaps_EnsMean_0001419123.zarr'), mode='w')

In [9]:
for iiter in ntimes[0,dd+1:]:
    for nmemb in nmembs:
        
        dsU = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                      % (nmemb)), 
                              grid_dir=gdir,
                              prefix=['diagUm'], delta_t=2e2,
                              iters=iiter, 
                              ignore_unknown_vars=True
                             ).reset_coords(drop=True).drop_dims(['YG','XC','Zl','Zu','Zp1']
                                                                ).chunk({'YC':ychunk,'XG':xchunk,
                                                                         'Z':10}).isel(time=0)
        dsV = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                      % (nmemb)), 
                              grid_dir=gdir,
                              prefix=['diagVm'], delta_t=2e2,
                              iters=iiter, 
                              ignore_unknown_vars=True
                             ).reset_coords(drop=True).drop_dims(['YC','XG','Zl','Zu','Zp1']
                                                                ).chunk({'YG':ychunk,'XC':xchunk,
                                                                         'Z':10}).isel(time=0)
        
        if nmemb == 100:
            TOTUTEND = dsU.TOTUTEND
            TOTVTEND = dsV.TOTVTEND
            Um_Advec = dsU.Um_Advec
            Vm_Advec = dsV.Vm_Advec
        else:
            TOTUTEND = xr.concat([TOTUTEND, dsU.TOTUTEND
                                 ], dim='nmemb')
            TOTVTEND = xr.concat([TOTVTEND, dsV.TOTVTEND
                                 ], dim='nmemb')
            Um_Advec = xr.concat([Um_Advec, dsU.Um_Advec
                                 ], dim='nmemb')
            Vm_Advec = xr.concat([Vm_Advec, dsV.Vm_Advec
                                 ], dim='nmemb')

        
    TOTUTEND.coords['nmemb'] = ('nmemb',nmembs)
    TOTVTEND.coords['nmemb'] = ('nmemb',nmembs)
    Um_Advec.coords['nmemb'] = ('nmemb',nmembs)
    Vm_Advec.coords['nmemb'] = ('nmemb',nmembs)
    
    dsave = TOTUTEND.mean('nmemb',skipna=True).to_dataset(name='TOTUTEND')
    dsave['TOTVTEND'] = TOTVTEND.mean('nmemb',skipna=True)
    dsave['Um_Advec'] = Um_Advec.mean('nmemb',skipna=True)
    dsave['Vm_Advec'] = Vm_Advec.mean('nmemb',skipna=True)
    dsave['TOTUTEND'].encoding = {}
    dsave['TOTVTEND'].encoding = {}
    dsave['Um_Advec'].encoding = {}
    dsave['Vm_Advec'].encoding = {}
    dsave.to_zarr(op.join(spec,'Snaps_Tot-Adv_EnsMean/%010d.zarr' % iiter), mode='w')
    dsave.close()
    
    dsU.close(); dsV.close()
    del TOTUTEND, TOTVTEND, Um_Advec, Vm_Advec
    
    print(iiter)

1529280
1531440
1533600
1535760
1537920
1540080
1542240
1544400
1546560
1548720
1550880
1553040
1555200
1557360
1559520
1561680
1563840
1566000
1568160
1570320
1572480
1574640
